In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
train_data = torchvision.datasets.MNIST('data',transform=transforms.ToTensor(),train=True)
test_data = torchvision.datasets.MNIST('data',transform=transforms.ToTensor(),train=False)

## Making Dataset Iteratable

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=100,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=100,shuffle=False)

## Building CNN class

In [17]:
class CNN(nn.Module):
    
    def __init__(self):
        
        super(CNN,self).__init__()
        
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,padding=2),nn.ReLU(),nn.MaxPool2d(kernel_size=2))
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,padding=2),nn.ReLU(),nn.MaxPool2d(kernel_size=2))
        self.fc1 = nn.Linear(32*7*7,10)
        
    def forward(self,X):
        
        out = self.layer1(X)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return out

In [18]:
model = CNN().to(device)

## Defining loss function and optimizers

In [28]:
loss_func = torch.nn.CrossEntropyLoss()
adam = torch.optim.Adam(model.parameters(),lr =0.001)

## Trainning

In [29]:
for epoc in range(0,5):
    
    for i,(image,label) in enumerate(train_loader):
        
        image = image.to(device)
        label = label.to(device)
        
        out = model.forward(image)
        loss = loss_func(out,label)
        
        adam.zero_grad()
        loss.backward()
        adam.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoc+1, 5, i+1, len(train_loader), loss.item()))
        
    

Epoch [1/5], Step [100/600], Loss: 2.2913
Epoch [1/5], Step [200/600], Loss: 2.2976
Epoch [1/5], Step [300/600], Loss: 2.3071
Epoch [1/5], Step [400/600], Loss: 2.3021
Epoch [1/5], Step [500/600], Loss: 2.3017
Epoch [1/5], Step [600/600], Loss: 2.2996
Epoch [2/5], Step [100/600], Loss: 2.3035
Epoch [2/5], Step [200/600], Loss: 2.3027
Epoch [2/5], Step [300/600], Loss: 2.2936
Epoch [2/5], Step [400/600], Loss: 2.3096
Epoch [2/5], Step [500/600], Loss: 2.2986
Epoch [2/5], Step [600/600], Loss: 2.2972
Epoch [3/5], Step [100/600], Loss: 2.2982
Epoch [3/5], Step [200/600], Loss: 2.2978
Epoch [3/5], Step [300/600], Loss: 2.2970
Epoch [3/5], Step [400/600], Loss: 2.3008
Epoch [3/5], Step [500/600], Loss: 2.3014
Epoch [3/5], Step [600/600], Loss: 2.2995
Epoch [4/5], Step [100/600], Loss: 2.2978
Epoch [4/5], Step [200/600], Loss: 2.3010
Epoch [4/5], Step [300/600], Loss: 2.2951
Epoch [4/5], Step [400/600], Loss: 2.3052
Epoch [4/5], Step [500/600], Loss: 2.2972
Epoch [4/5], Step [600/600], Loss:

In [30]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 11.35 %
